In [1]:
%gui qt
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point

## Make a napari viewer...

In [13]:
v = napari.Viewer()

## Load cell x gene data from 1 dataset

In [14]:
all_cells = pd.read_csv("/Users/brian/Downloads/MERFISH_dataset1_slice241.csv")

all_cells.head(5)

,index,cell_volume,pos_x,pos_y,1700022I11Rik,1810046K07Rik,5031425F14Rik,5730522E02Rik,Acta2,Adam2,...,Gad2,Tac2,Lamp5,Cnr1,Pvalb,Sst,Rab3b,Slc17a7,Penk,Gad1
0,f7e6e078-844d-48ba-a29b-0773d387b492,361.010125,7632.514829,72.431533,0.0,0.0,0.0,0.0,0.0,0.0,...,0.195170,0.152147,0.077308,0.134957,0.061733,0.124043,0.154878,0.141351,0.064708,0.074285
1,ab895fc9-89c9-4697-a5da-02de471c63c8,1342.752007,7616.930382,56.276356,0.0,0.0,0.0,0.0,0.0,0.0,...,0.533478,0.281789,0.057863,0.115466,0.092632,0.174297,0.195167,0.260387,0.061333,0.063949
2,27b7cdc0-4d6f-4993-adf4-44c587ddf4e7,175.630882,7613.150777,-6.374358,0.0,0.0,0.0,0.0,0.0,0.0,...,0.311116,0.085166,0.039396,0.061605,0.059481,0.047501,0.047976,0.301797,0.027551,0.037051
3,0001439e-700a-4c9e-ba58-28e57fc1ddd0,1665.116209,7612.373659,73.962139,0.0,0.0,0.0,0.0,0.0,0.0,...,0.467406,0.278138,0.063089,0.116956,0.101446,0.197969,0.240243,0.391096,0.118575,0.093829
4,9b63af45-4299-4fef-b4b9-ab9213345fbf,195.804820,7599.320544,178.496818,0.0,0.0,0.0,0.0,0.0,0.0,...,0.327333,0.230570,0.049680,0.096590,0.098840,0.095372,0.117938,0.093033,0.360200,0.046835


### Add the cells as points, double-plotting cells with high Rorb, Cux2 if available

In [15]:
cells = v.add_points(all_cells[["pos_x","pos_y"]], symbol='o', name = "all", opacity = 0.8, blending = "translucent")
colorlist = ["magenta", "green","yellow","cyan", "blue", "orange"]
for ii, gene_to_plot in enumerate(["Cux2", "Atp1a3" , "Vxn", "Pcp4" , "Fezf2" , "Rorb" , "Gm11549"]):

    if gene_to_plot in all_cells:
        cux2condition = all_cells[gene_to_plot]>0.5*all_cells[gene_to_plot].median()
        cux2cells = v.add_points(all_cells[cux2condition][["pos_x","pos_y"]], symbol='disc',
                                 name = gene_to_plot+"+",edge_color= "black", size = 10,
                                 face_color= colorlist[ ii % len(colorlist)] , blending = "translucent", opacity = 0.3)


### annotate the layers in that viewer in a single `shapes` layer
#### Naming/order Convention:
0. outline of entire column as rectangle
1. Layer 1
1. Layer 2/3
2. Layer 4
3. Layer 5
4. Layer 6
5. single line segment, pointing from pia to white matter 

#### annotate the layer locations in a list

In [18]:
v.layers

[<Points layer 'all' at 0x12f80e210>,
 <Points layer 'Cux2+' at 0x130345a50>,
 <Points layer 'Fezf2+' at 0x12fc51090>,
 <Points layer 'Rorb+' at 0x1306e2d50>,
 <Shapes layer 'Shapes' at 0x1307fa510>]

In [19]:
is_in_layer = {ii:[Polygon(p).intersects(Point(a))  for a  in v.layers[0].data] for ii,p in enumerate(v.layers[-1].data[:-1])}

In [ ]:
# ok:
# 1. make a couple functions to do this
#.  #. do a spatially partitioned search based on bounding box of polygon
# 2. throw annotation into separate vector matched to point cloud
#. 

In [58]:
# depth coordinate calculation

# dot product with normalized cortical column vector:

cvec = v.layers[-1].data[-1][1,:] - v.layers[-1].data[-1][0,:]
cvec_norm = cvec/np.linalg.norm(cvec,2)
depth = np.dot(np.column_stack([all_cells["pos_x"]-cvec[0],all_cells["pos_y"]-cvec[1]]),cvec_norm)

# norm?



In [61]:
depth.min()

689.9134886560516

In [56]:
e1 = [.5, 0.5]
ps = np.array([[0,1231],[23,444], [2,-2]])
np.dot( ps, cvec_norm)

array([1150.82491286,  423.24703813,   -1.15975915])

In [57]:
cvec_norm

array([0.35499038, 0.93486995])

In [26]:
all_cells["layer"] = np.zeros_like(all_cells["pos_x"])-1
layer_keys=[None,"L1","L2/3","L4","L5","L6"]
for ii in is_in_layer.keys():
    all_cells["layer"][is_in_layer[ii]] = layer_keys[ii]

/Users/brian/anaconda2/envs/starfish/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [25]:
all_cells

,index,cell_volume,pos_x,pos_y,1700022I11Rik,1810046K07Rik,5031425F14Rik,5730522E02Rik,Acta2,Adam2,...,Tac2,Lamp5,Cnr1,Pvalb,Sst,Rab3b,Slc17a7,Penk,Gad1,layer
0,f7e6e078-844d-48ba-a29b-0773d387b492,361.010125,7632.514829,72.431533,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.152147,0.077308,0.134957,0.061733,0.124043,0.154878,0.141351,0.064708,0.074285,0
1,ab895fc9-89c9-4697-a5da-02de471c63c8,1342.752007,7616.930382,56.276356,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.281789,0.057863,0.115466,0.092632,0.174297,0.195167,0.260387,0.061333,0.063949,0
2,27b7cdc0-4d6f-4993-adf4-44c587ddf4e7,175.630882,7613.150777,-6.374358,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.085166,0.039396,0.061605,0.059481,0.047501,0.047976,0.301797,0.027551,0.037051,0
3,0001439e-700a-4c9e-ba58-28e57fc1ddd0,1665.116209,7612.373659,73.962139,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.278138,0.063089,0.116956,0.101446,0.197969,0.240243,0.391096,0.118575,0.093829,0
4,9b63af45-4299-4fef-b4b9-ab9213345fbf,195.804820,7599.320544,178.496818,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.230570,0.049680,0.096590,0.098840,0.095372,0.117938,0.093033,0.360200,0.046835,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3573,eeb4082f-f435-4b6d-abfb-5019682c3981,2972.189573,6272.040622,1081.832081,0.0,0.000000,0.0,0.0,0.288905,0.0,...,0.737469,0.989460,0.168662,0.243034,0.139443,0.725628,9.973541,0.328750,0.000000,0
3574,cd7163a7-e0a4-4c10-be19-56953250b197,1802.341760,6250.696165,1173.508265,0.0,0.000000,0.0,0.0,0.476424,0.0,...,0.546792,0.370871,0.384851,0.228930,0.028178,0.525361,7.538988,0.141847,0.044894,0
3575,4c24ab01-9fb3-4e82-b299-876e7d82051d,1915.971644,6091.278160,927.265594,0.0,0.448169,0.0,0.0,0.000000,0.0,...,0.673126,0.619532,0.058533,0.236143,0.175464,0.745498,6.898695,0.266796,0.000000,0
3576,754e3ac7-4b76-4112-97be-29542f3129e4,2509.944417,6208.623008,761.450600,0.0,0.000000,0.0,0.0,0.684222,0.0,...,0.218305,0.043328,0.061520,0.064992,0.152001,0.178986,0.328401,0.103497,0.037573,0
